### Реализовать SVM и Logistic Regression для данного датасета: https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original) Метрики оценки: Accuracy, Precision, Recall, F1-Score

In [172]:
import pandas as pd

df = pd.read_csv("breast_cancer/wisconsin.data")
df.columns = [
    'Id',
    'Clump Thickness', # толщина комков
    'Uniformity of Cell Size', # однородность размера клетки
    'Uniformity of Cell Shape', # однородность формы клетки
    'Marginal Adhesion', # предельная адгезия
    'Single Epithelial Cell Size', # размер одной эпителиальной клетки
    'Bare Nuclei', # голые ядра
    'Bland Chromatin', # мягкий хроматин
    'Normal Nucleoli', # нормальные ядра
    'Mitoses', # митозы
    'Class' # 2 - доброкачественные, 4 - злокачественные"
]

# удаляем колонку идентификаторов - они не нужны в вычислениях
df.drop('Id', axis=1, inplace=True)

# в датасете есть 16 записей у которых отсутствует значение аттрибута Bare Nuclei - ими можно пренебречь:
df = df[df['Bare Nuclei'] != '?'].astype({'Bare Nuclei':'int64'})

# заменяем 2 и 4 на 0 и 1 для класса
df['Class'] = df['Class'].apply(lambda x: 1 if x == 4 else 0)

# все значения х-к находятся в интервале 0:10
df.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,4,4,5,7,10,3,2,1,0
1,3,1,1,1,2,2,3,1,1,0
2,6,8,8,1,3,4,3,7,1,0
3,4,1,1,3,2,1,3,1,1,0
4,8,10,10,8,7,10,9,7,1,1


In [173]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.25)

x_train = train.iloc[:, 0:9].values
y_train = train.iloc[:, 9].values

x_test = test.iloc[:, 0:9].values
y_test = test.iloc[:, 9].values

In [174]:
from sklearn import metrics

def print_metrics(y_pred, y_real):
    accuracy = metrics.accuracy_score(y_real, y_pred)
    precision = metrics.precision_score(y_real, y_pred)
    recall = metrics.recall_score(y_real, y_pred)
    f1 = metrics.f1_score(y_real, y_pred)
    
    print("Accuracy =", float("%0.2f" % accuracy))
    print("Precision =", float("%0.2f" % precision))
    print("Recall =", float("%0.2f" % recall))
    print("F1 =", float("%0.2f" % f1))

In [175]:
def fit_and_predict(model):
    model.fit(x_train, y_train)
    
    return model.predict(x_test)

## Метод опорных векторов

In [176]:
model = sklearn.svm.SVC(kernel='linear')

y_pred = fit_and_predict(model)

print_metrics(y_pred, y_test)

Accuracy = 0.95
Precision = 0.9
Recall = 0.96
F1 = 0.93


## Логистическая регрессия

In [177]:
model = sklearn.linear_model.LogisticRegression(solver='lbfgs')

y_pred = fit_and_predict(model)

print_metrics(y_pred, y_test)

Accuracy = 0.95
Precision = 0.9
Recall = 0.96
F1 = 0.93
